# Evaluate

In [32]:
import os
import torch
import numpy as np

# move to root dir
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

from torch.utils.data import DataLoader
from data.dataloader import HAM10000
from model.models import ImgClassificationModel

from glob import glob
from gui.ensemble import ens_predict as ens_predict
from tqdm import tqdm

In [33]:
# test dataset
test_ds = HAM10000('input/aug/test')
test_dl = DataLoader(test_ds, batch_size=16, num_workers=12)

In [34]:
# output files
output_mcd_file = 'output/predictions/monte_carlo.npy'

## Monte-Carlo

In [35]:
from model.monte_carlo import predict_mcd_class

In [36]:
checkpoint      = 'output/checkpoints/resnet-drop_epoch=13_val_acc=0.80.ckpt'
device          = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ImgClassificationModel.load_from_checkpoint(checkpoint)
model = model.to(device)

In [37]:
# do predictions
pred = []; true = []
for Xs, ys in tqdm(test_dl):
    Xs = Xs.to(device)
    pred += list(predict_mcd_class(model, Xs, num_samples=100))
    true += list(ys.numpy())

# create np array of shape (n, 2) - each row being [pred:true]
output_mcd = np.array([pred, true])
output_mcd = output_mcd.reshape(-1, 2)

# save to file
output_mcd_file = 'output/predictions/monte_carlo.npy'
np.save(output_mcd_file, output_mcd)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [04:45<00:00,  6.07s/it]


## Ensemble

In [38]:
model_names    = ['resnet-drop', 'resnet', 'alexnet', 'vgg', 'densenet', ] # 'squeezenet', ]
checkpoint_dir = 'output/checkpoints'
checkpoints    = {m: glob(f'{checkpoint_dir}/{m}_*')[0] for m in model_names}

# load models
device = torch.device('cuda:0')
models = {m: ImgClassificationModel.load_from_checkpoint(checkpoints[m]).to(device)
          for m in model_names}

In [39]:
pred = []; true = []


for Xs, ys in tqdm(test_dl):
    Xs = Xs.to(device)
    p_list = []
    for X in Xs:

        p = ens_predict(X.view([1,3,224,224]))
        p_list.append(p)

    pred += list(p_list)
    true += list(ys.numpy())


output_ens = np.array([pred, true])
output_ens = output_ens.reshape(-1, 2)

# save to file
output_ens_file = 'output/predictions/ensemble.npy'
np.save(output_ens_file, output_ens)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:18<00:00,  2.48it/s]


## Calculate Metrics

In [45]:
from sklearn.metrics import f1_score, accuracy_score

In [46]:
output_mcd = np.load(output_mcd_file)

In [47]:
acc = accuracy_score(output_mcd[:,1], output_mcd[:,0])
f1  = f1_score(output_mcd[:,1], output_mcd[:,0], average='macro')

print(f'Monte Carlo - Acc: {acc:0.2}, F1: {f1:0.2}')

Monte Carlo - Acc: 0.87, F1: 0.75


In [48]:
output_ens = np.load(output_ens_file)

In [49]:
output_mcd = np.load(output_ens_file)
acc = accuracy_score(output_ens[:,1], output_ens[:,0])
f1  = f1_score(output_ens[:,1], output_ens[:,0], average='macro')

print(f'Ensemble Model - Acc: {acc:0.2}, F1: {f1:0.2}')

Ensemble Model - Acc: 0.89, F1: 0.77
